In [1]:
from dariah_topics import preprocessing as pre
from dariah_topics import visualization as visual
from dariah_topics import mallet as mal
from dariah_topics import evaluation

/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationW

## Preprocessing

#### Liste mit Dateinamen erzeugen

In [2]:
path_txt = "corpus_txt"
#path_txt = "grenzbote_plain/*/"

doclist_txt = pre.create_document_list(path_txt)
assert doclist_txt, "No documents found"
doclist_txt[:5]

['corpus_txt/Poe_EurekaAProsePoem.txt',
 'corpus_txt/Howard_TheDevilinIron.txt',
 'corpus_txt/Lovecraft_TheShunnedHouse.txt',
 'corpus_txt/Howard_SchadowsinZamboula.txt',
 'corpus_txt/Doyle_AStudyinScarlet.txt']

In [3]:
path_csv = "corpus_csv"

doclist_csv = pre.create_document_list(path_csv, 'csv')
doclist_csv[:5]

['corpus_csv/Howard_GodsoftheNorth.txt.csv',
 'corpus_csv/Poe_EurekaAProsePoem.txt.csv',
 'corpus_csv/Poe_TheMasqueoftheRedDeath.txt.csv',
 'corpus_csv/Poe_ThePurloinedLetter.txt.csv',
 'corpus_csv/Howard_ShadowsintheMoonlight.txt.csv']

#####  Liste mit Dokumentenlabels erzeugen - (Funktion wird durch Thorsten's generischere Funktion ersetzt)

In [4]:
doc_labels = list(pre.get_labels(doclist_txt))
doc_labels[:5]

['corpus_txt/Poe_EurekaAProsePoem.txt',
 'corpus_txt/Howard_TheDevilinIron.txt',
 'corpus_txt/Lovecraft_TheShunnedHouse.txt',
 'corpus_txt/Howard_SchadowsinZamboula.txt',
 'corpus_txt/Doyle_AStudyinScarlet.txt']

#### Corpus laden

In [5]:
corpus_txt = pre.read_from_txt(doclist_txt)

In [6]:
corpus_csv = pre.read_from_csv(doclist_csv)

#### Tokenisieren

In [7]:
doc_tokens = [list(pre.tokenize(txt)) for txt in list(corpus_txt)]
doc_tokens[0][:5]

['eureka', 'prose', 'poem', 'by', 'edgar']

#### Create Dictionaries

In [8]:
id_types, doc_ids = pre.create_dictionaries(doc_labels, doc_tokens)

#### Sparse BOW Model

In [9]:
sparse_bow = pre.create_mm(doc_labels, doc_tokens, id_types, doc_ids)

In [10]:
sparse_bow[:5]

0
doc_id token_id   
1      8192      1
       8194      1
       6827      3
       5         1
       8202      3

#### Convert to Doc-Topic Matrix

In [11]:
'''import pandas as pd

doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

cols = [doc2id[key] for key in set(sparse_bow.index.get_level_values("doc_id"))]
#idx = [type2id[key] for key in set(sparse_bow.index.get_level_values("token_id"))]

set(sparse_bow.index.get_level_values("token_id"))

#doctopic_matrix = pd.DataFrame(columns=cols, index=idx)'''

'import pandas as pd\n\ndoc2id = {value : key for key, value in doc_ids.items()}\ntype2id = {value : key for key, value in id_types.items()}\n\ncols = [doc2id[key] for key in set(sparse_bow.index.get_level_values("doc_id"))]\n#idx = [type2id[key] for key in set(sparse_bow.index.get_level_values("token_id"))]\n\nset(sparse_bow.index.get_level_values("token_id"))\n\n#doctopic_matrix = pd.DataFrame(columns=cols, index=idx)'

#### Remove Features

In [12]:
import os.path
basepath = os.path.abspath('.')

with open(os.path.join(basepath, "tutorial_supplementals", "stopwords", "en.txt"), 'r', encoding = 'utf-8') as f: 
    stopword_list = f.read().split('\n')
    
stopword_list = set(stopword_list)

In [13]:
sparse_df_stopwords_removed = pre.remove_features(sparse_bow, id_types, stopword_list)

In [14]:
print(len(sparse_bow))
print(len(sparse_df_stopwords_removed))

50942
45754


##### Create Feature Remove Lists

In [15]:
stopword_list = pre.find_stopwords(sparse_bow, id_types, 100)
len(stopword_list)

100

In [16]:
hapax_list = pre.find_hapax(sparse_bow, id_types)
len(hapax_list)

17746

##### Remove Features

In [17]:
feature_list = set(stopword_list).union(hapax_list)
clean_term_frequency = pre.remove_features(sparse_bow, id_types, feature_list)

In [18]:
len(clean_term_frequency)

4892

## Topic Modeling with Gensim

##### Saving Sparse BOW

In [19]:
num_docs = max(sparse_bow.index.get_level_values("doc_id"))
num_types = max(sparse_bow.index.get_level_values("token_id"))
sum_counts = sum(sparse_bow[0])

header_string = str(num_docs) + " " + str(num_types) + " " + str(sum_counts) + "\n"

with open("gb_plain.mm", 'w', encoding = "utf-8") as f:
    pass

with open("gb_plain.mm", 'a', encoding = "utf-8") as f:
    f.write("%%MatrixMarket matrix coordinate real general\n")
    f.write(header_string)
    sparse_bow.to_csv( f, sep = ' ', header = None)

#### Import Market Matrix

In [20]:
from gensim.models import LdaModel
from gensim.corpora import MmCorpus

In [21]:
mm = MmCorpus("gb_plain.mm")

In [22]:
#in case you're only loading the corpus - build dict first

#import pickle

#pickle.dump( id_types, open( "gb_plain.dictionary", "wb" ) )

In [23]:
#id_types = pickle.load(open("gb_plain.dictionary", 'rb'))

In [24]:
#mm = gensim.corpora.MmCorpus("gb_all.mm")

In [25]:
#id_types = pickle.load(open("gb_all.dictionary", 'rb'))

In [26]:
doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

In [27]:
#model = LdaModel(corpus=mm, id2word=type2id, num_topics=60, alpha = "symmetric", passes = 10) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen
model = LdaModel(corpus=mm, id2word=type2id, num_topics=10)

WARNING gensim.models.ldamodel: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


In [37]:
#Anzeige der keywords für topic n
n = 1
topic_nr_x = model.get_topic_terms(n)

topicTerms = [type2id[i[0]] for i in topic_nr_x]
topicTerms

['grasses',
 'quench',
 'replacement',
 "creature's",
 'butterflies',
 'flagship',
 'struggles',
 'cow-kick',
 'subterrene',
 'swam']

In [40]:
#Anzeige der id des ersten keywords aus dem topic
'''n = 0
for key, value in type2id.items():
    if value == topicTerms[n]:
        print(key)'''

19964


In [41]:
#Gensim-Funktion -> Anzeige des word_id-probability-mappings eines topics n
'''n = 0
topicN = model.get_topic_terms(n)
topicN'''

[(19964, 0.045547697399961583),
 (7484, 0.02555935157378144),
 (20844, 0.021748152184078273),
 (12390, 0.018450207185275385),
 (17775, 0.014793747674321884),
 (2813, 0.010719961603461841),
 (17527, 0.010692766393465185),
 (10804, 0.0090416682617215326),
 (7691, 0.0087376172189662857),
 (901, 0.008659425952339133)]

In [32]:
#Topic probabilities
topics = model.show_topics(num_topics = 10)
topics[:2]

[(0,
  '0.046*"grasses" + 0.026*"replacement" + 0.022*"quench" + 0.018*"creature\'s" + 0.015*"struggles" + 0.011*"flagship" + 0.011*"butterflies" + 0.009*"swam" + 0.009*"cow-kick" + 0.009*"packs"'),
 (1,
  '0.074*"grasses" + 0.028*"quench" + 0.026*"replacement" + 0.023*"creature\'s" + 0.016*"butterflies" + 0.016*"flagship" + 0.015*"struggles" + 0.011*"cow-kick" + 0.011*"subterrene" + 0.010*"swam"')]

In [51]:
from scipy.io import mmread
test = mmread("gb_plain.mm")
import pandas as pd
pd.DataFrame(test.data)

,0
0,1.0
1,1.0
2,3.0
3,1.0
4,3.0
5,1.0
6,3.0
7,2.0
8,1.0
9,4.0


## Topic Modeling mit Mallet

##### Mallet Binary erzeugen

In [42]:
'''import os

#Variablen zum Testen
path_to_mallet = "/home/sina/Uni/Dariah/mallet/bin/mallet"

path_to_corpus = os.path.join(os.path.abspath('.'), 'corpus_txt')
assert os.path.exists(path_to_corpus)

malletBinary = mal.create_mallet_binary(path_to_corpus, path_to_mallet)
malletBinary'''

'tutorial_supplementals/mallet_output/malletBinary.mallet'

##### Mallet output erzeugen

In [43]:
'''outfolder = os.path.join(os.path.abspath('.'), "tutorial_supplementals/mallet_output")
mal.create_mallet_model(malletBinary,outfolder)'''

##### Matrix von Mallet output erzeugen

In [44]:
'''basepath = os.path.join(os.path.abspath('.'), "tutorial_supplementals/mallet_output")
doc_topics = os.path.join(basepath, "doc_topics.txt")'''

In [45]:
'''doctopicsMatrix = mal.create_MalletMatrix(doc_topics)'''

In [46]:
'''import pandas as pd
pd.DataFrame(doctopicsMatrix)'''

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.018470,0.012259,0.069794,0.008336,0.023374,0.380353,0.007028,0.088754,0.003759,0.058352,0.008336,0.037757,0.009971,0.000817,0.004740,0.017816,0.145636,0.092677,0.003432,0.008336
1,0.022766,0.020562,0.091914,0.011052,0.015209,0.158291,0.004125,0.098274,0.019932,0.078248,0.006140,0.026922,0.006266,0.001984,0.001669,0.209302,0.142925,0.075225,0.004755,0.004440
2,0.007416,0.019088,0.106867,0.002845,0.279410,0.115718,0.003331,0.086636,0.016705,0.053227,0.007513,0.023902,0.005763,0.000559,0.001094,0.013398,0.170622,0.079974,0.003039,0.002894
3,0.015372,0.026422,0.080110,0.217192,0.031297,0.138739,0.003087,0.082710,0.014202,0.079981,0.006207,0.022197,0.005882,0.001722,0.001852,0.016737,0.179948,0.068151,0.004062,0.004127
4,0.002207,0.536885,0.131463,0.004098,0.005990,0.002207,0.072194,0.012926,0.030580,0.053909,0.005990,0.065259,0.002207,0.005359,0.006620,0.007251,0.041299,0.006620,0.001576,0.005359
5,0.009872,0.107346,0.168356,0.005603,0.001512,0.011473,0.377535,0.016987,0.018410,0.079420,0.005425,0.112327,0.007560,0.002935,0.004358,0.005069,0.053273,0.011117,0.000445,0.000978
6,0.005495,0.237705,0.169789,0.003513,0.003873,0.008377,0.249955,0.018465,0.021348,0.070528,0.007836,0.108719,0.002252,0.009638,0.008557,0.003333,0.056837,0.010178,0.002072,0.001531
7,0.003606,0.135263,0.164109,0.002311,0.000462,0.015440,0.328310,0.029863,0.047245,0.055936,0.012666,0.100314,0.001942,0.006749,0.019323,0.001387,0.057045,0.014885,0.001757,0.001387
8,0.404977,0.023547,0.086005,0.007515,0.007849,0.042585,0.008183,0.049933,0.013527,0.122077,0.006847,0.022879,0.007849,0.016199,0.006847,0.003841,0.136774,0.026887,0.003173,0.002505
9,0.011169,0.058814,0.063583,0.002117,0.000365,0.010877,0.003236,0.009904,0.003042,0.232115,0.002166,0.005816,0.000511,0.419773,0.001192,0.001679,0.164858,0.005086,0.002798,0.000900


## Visualisierung 

In [ ]:
'''lda_model = 'out_easy/corpus.lda'
corpus = 'out_easy/corpus.mm'
dictionary = 'out_easy/corpus.dict'
doc_labels = 'out_easy/corpus_doclabels.txt'
interactive  = False

vis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)'''

#### Heatmap

In [ ]:
#heatmap = visual.make_heatmap()

In [ ]:
#visual.save_heatmap("./visualizations/heatmap")

#### Interactive

In [ ]:
#vis = collection.Visualization(lda_model, corpus, dictionary, doc_labels, interactive=True)

In [ ]:
#vis.make_interactive()

In [ ]:
#vis.save_interactive("./visualizations/interactive")

## Evaluation

In [ ]:
segmented_topics = evaluation.topic_segmenter(model, type2id, 60)
segmented_topics[0][:5]

In [ ]:
umass_probability = evaluation.token_probability(doc_tokens, segmented_topics)
umass_probability

In [ ]:
umass = evaluation.calculate_umass(segmented_topics, umass_probability, doc_tokens, 60)
umass

In [ ]:
wiki_list = evaluation.wikipedia_table_crawler()

In [ ]:
wiki_corpus = evaluation.wikipedia_crawler(wiki_list, size=5)

In [ ]:
uci_probability = evaluation.token_probability(wiki_corpus, segmented_topics)

In [ ]:
pmi = evaluation.calculate_pointwise_mutual_information(segmented_topics, wiki_corpus, uci_probability)

In [ ]:
uci = evaluation.calculate_uci(pmi, wiki_corpus, 60)
uci

![success](http://cdn2.hubspot.net/hub/128506/file-446943132-jpg/images/computer_woman_success.jpg)